# Real-Time Streaming Fraud Detection Pipeline

This notebook demonstrates an end-to-end streaming fraud detection pipeline that combines stateless and stateful features.

## Features

**Stateless Features (~40 columns):**
- Time-based: hour, day, business hours, cyclical encodings
- Amount-based: log, sqrt, categories, z-scores
- Merchant: risk scores based on category
- Location: risk indicators, region classification
- Device: device type detection
- Network: IP classification

**Stateful Features (~15 columns):**
- Velocity: transaction counts in time windows (10 min, 1 hour)
- IP tracking: IP change detection and counts
- Location anomalies: impossible travel detection (velocity > 800 km/h)
- Amount anomalies: z-score calculation vs user history
- Fraud scoring: composite 0-100 score with prediction flag

## Architecture

```
Streaming Source (rate)
    ↓
Generate Transactions (synthetic data)
    ↓
Apply Stateless Features (AdvancedFeatureEngineering)
    ↓
Apply Stateful Fraud Detection (transformWithState)
    ↓
Write to Lakebase PostgreSQL in Realtime Mode (foreach)
```

## Prerequisites

- Run `00_setup.ipynb` first to create `transaction_features` table
- Databricks Runtime 17.3+ (with Spark 4.0+ for transformWithStateInPandas)
- Lakebase PostgreSQL instance provisioned

## Output

All features (stateless + stateful) are written to:
- **Table**: `transaction_features` (~20+ columns)
- **Write latency**: 400ms

In [0]:
dbutils.library.restartPython()
spark.conf.set("spark.sql.streaming.stateStore.providerClass", "com.databricks.sql.streaming.state.RocksDBStateStoreProvider")
spark.conf.set("spark.databricks.streaming.statefulOperator.asyncCheckpoint.enabled", "false")
spark.conf.set("spark.sql.streaming.stateStore.rocksdb.changelogCheckpointing.enabled", "true")
spark.conf.set("spark.sql.shuffle.partitions", "4")

In [0]:
# Import required libraries
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pandas as pd
import logging

# Import utility modules
from utils.data_generator import TransactionDataGenerator
from utils.feature_engineering import (
    AdvancedFeatureEngineering, 
    FraudDetectionFeaturesProcessor,
    get_fraud_detection_output_schema
)
from utils.lakebase_client import LakebaseClient

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

INFO:dbldatagen._version:Version : VersionInfo(major='0', minor='4', patch='0', release='', build='')


## Step 1: Configuration

Configure Lakebase connection and initialize components.

In [0]:
# Lakebase connection configuration
LAKEBASE_CONFIG = {
    "instance_name": "rtm-lakebase-demo",
    "database": "databricks_postgres"
}

# Initialize components
data_gen = TransactionDataGenerator(spark)
feature_engineer = AdvancedFeatureEngineering(spark)
lakebase = LakebaseClient(**LAKEBASE_CONFIG)

# Test Lakebase connection
if lakebase.test_connection():
    print("Connected to Lakebase PostgreSQL")
else:
    raise Exception("Failed to connect to Lakebase")

# Verify transaction_features table exists
print("\nVerifying transaction_features table...")
try:
    stats = lakebase.get_table_stats("transaction_features")
    print(f"  Table exists with {stats['total_rows']:,} rows")
except Exception as e:
    print("  Table not found. Please run 00_setup.ipynb first!")
    raise Exception("transaction_features table does not exist")

0.68.0


INFO:utils.lakebase_client:Lakebase connection test successful


Connected to Lakebase PostgreSQL

Verifying transaction_features table...
0.68.0


INFO:utils.lakebase_client:Table stats: 865 rows


  Table exists with 865 rows


## Step 2-A: Generate Streaming Transaction Data

Create a streaming source that continuously generates synthetic transactions.

In [0]:
# Generate streaming transaction data
df_transactions = data_gen.generate_transaction_data(
    num_users=1000,           #1000 unique users
    num_merchants=1000,      # 1000 unique merchants
    rows_per_second=100      # 100 transactions per second
)

INFO:utils.data_generator:Creating synthetic transaction source with dbldatagen...
INFO:utils.data_generator:   Rows: 1000
INFO:utils.data_generator:   Users: 1000, Merchants: 1000
INFO:utils.data_generator:Synthetic source created successfully


## Step 2-B: Write Stream Data to Kafka Topic

In [0]:
# Retrieve secrets from Databricks secret scope
KAFKA_KEY = dbutils.secrets.get(scope = "jaypalaniappan", key = "KAFKA_KEY")
KAFKA_SECRET = dbutils.secrets.get(scope = "jaypalaniappan", key = "KAFKA_SECRET")
KAFKA_SERVER = dbutils.secrets.get(scope = "jaypalaniappan", key = "KAFKA_SERVER")
KAFKA_TOPIC = "transactions_source_stream"

#Generator Configuration
CHECKPOINT_LOCATION = "/tmp/transaction-data-generator-checkpoint"
dbutils.fs.rm(CHECKPOINT_LOCATION, True) #remove old checkpoint

#Convert row to JSON String
json_df = df_transactions.select(to_json(struct(*[col(c) for c in df_transactions.columns])).alias("value"), col("transaction_id").alias("key"))

kafkaWriter = (
    json_df
    .writeStream
    .format("kafka")
    .option("kafka.bootstrap.servers", KAFKA_SERVER)
    .option("kafka.security.protocol", "SASL_SSL")
    .option("kafka.sasl.jaas.config", f"kafkashaded.org.apache.kafka.common.security.plain.PlainLoginModule required username='{KAFKA_KEY}' password='{KAFKA_SECRET}';")
    .option("kafka.ssl.endpoint.identification.algorithm", "https")
    .option("kafka.sasl.mechanism", "PLAIN")
    .option("topic", KAFKA_TOPIC)
    .option("failOnDataLoss", "true")
    .option("checkpointLocation", CHECKPOINT_LOCATION)
)

kafkaQuery = kafkaWriter.start()

print(f"""
[STREAMING] Kafka producer started successfully
  Query ID: {kafkaQuery.id}
  Query Name: {kafkaQuery.name}
  Status: {kafkaQuery.status}
  Kafka Topic: {KAFKA_TOPIC}
  Checkpoint: {CHECKPOINT_LOCATION}
  Serialization: Protobuf (Confluent Schema Registry)
""")


[STREAMING] Kafka producer started successfully
  Query ID: 9b556d07-caa9-4f40-89af-04858efbb254
  Query Name: None
  Status: {'message': 'Initializing sources', 'isDataAvailable': False, 'isTriggerActive': False}
  Kafka Topic: transactions_source_stream
  Checkpoint: /tmp/transaction-data-generator-checkpoint
  Serialization: Protobuf (Confluent Schema Registry)



## Step 3-A: Read Data from Kafka Topic

In [0]:
value_schema = "STRUCT<transaction_id: STRING, user_id: STRING, merchant_id: STRING, amount: DOUBLE, currency: STRING, merchant_category: STRING, payment_method: STRING, ip_address: STRING, device_id: STRING, latitude: DOUBLE, longitude: DOUBLE, card_type: STRING, timestamp: TIMESTAMP>"

kafka_df = (
    spark
    .readStream
    .format("kafka")
    .option("kafka.bootstrap.servers", KAFKA_SERVER)
    .option("kafka.security.protocol", "SASL_SSL")
    .option("kafka.sasl.jaas.config", f"kafkashaded.org.apache.kafka.common.security.plain.PlainLoginModule required username='{KAFKA_KEY}' password='{KAFKA_SECRET}';")
    .option("kafka.ssl.endpoint.identification.algorithm", "https")
    .option("kafka.sasl.mechanism", "PLAIN")
    .option("subscribe", KAFKA_TOPIC)
    .option("startingOffsets", "latest")
    .option("failOnDataLoss", "false")  
    .load()
) \
.select(
    from_json(col("value").cast("string"), 
              value_schema).alias("value"), 
    col("timestamp").alias("kafka_timestamp")) \
.selectExpr("value.*", "kafka_timestamp") \
  .drop("timestamp") \
  .withColumnRenamed("kafka_timestamp", "timestamp")

#display(kafka_df)

## Step 3-B: Apply Stateless Features

Apply time-based, amount-based, merchant, location, device, and network features.

In [0]:
df_with_stateless_features = feature_engineer.apply_all_features(kafka_df)

INFO:utils.feature_engineering:Applying streaming-compatible feature engineering...
INFO:utils.feature_engineering:Creating time-based features...
INFO:utils.feature_engineering:Creating amount-based features...
INFO:utils.feature_engineering:Creating merchant features (streaming-only)...
INFO:utils.feature_engineering:Skipping stateless location features (optimized out)
INFO:utils.feature_engineering:Skipping device features (optimized out)
INFO:utils.feature_engineering:Creating network features (streaming-only)...
INFO:utils.feature_engineering:Streaming feature engineering completed!


## Step 4: Apply Stateful Features

Use `transformWithStateInPandas` to maintain per-user state and detect fraud patterns.

In [0]:
# Apply stateful fraud detection using transformWithState
df_with_fraud_features = df_with_stateless_features \
    .withWatermark("timestamp", "1 minutes") \
    .groupBy("user_id") \
    .transformWithState(
        statefulProcessor=FraudDetectionFeaturesProcessor(),
        outputStructType=get_fraud_detection_output_schema(),
        outputMode="Update",
        timeMode="processingTime"
    )

## Step 5: Write to Lakebase

Stream all features to Lakebase for real-time serving.

In [0]:
dbutils.fs.rm('/tmp/fraud_pipeline_checkpoint', True)

True

In [0]:
#Get Schema from dataframe
table_schema = df_with_fraud_features.schema

#Initialize lakebase writer
lakebase_writer  = lakebase.get_foreach_writer(column_names=table_schema.names, batch_size=1)

#Start streaming query
query = df_with_fraud_features \
    .writeStream \
    .outputMode("update") \
    .foreach(lakebase_writer) \
    .option("checkpointLocation", "/tmp/fraud_pipeline_checkpoint") \
    .trigger(realTime="5 minutes") \
    .start()

0.68.0


## Step 6: Stop Streaming Query

Stop the streaming pipeline when done.

In [0]:
# # Stop streaming query
# if query.isActive:
#     query.stop()
#     print("Streaming query stopped")

# print("\nPipeline complete!")